# Scripts for preparing metatdata for HiQBind Dataset

In this script, we will create CSV files containing relevant metadata that will be used for the HiQBind workflow to process structures.

In [6]:
import pandas as pd
from tqdm import tqdm
import re
import requests
import math

from rdkit import Chem

In [7]:
def get_smiles_from_rcsb(comp_id: str):
    """
    Query ligand SMILES from RCSB

    Parameters
    ----------
    comp_id: str
        The ligand ID, usually a three-letter code
    
    Returns
    -------
    smi: str
        The SMILES of the query ligand. If fail to get, will return a vacant string
    """
    query = '''{chem_comp(comp_id: "%s") {
        rcsb_chem_comp_descriptor {
        SMILES_stereo SMILES InChI
        }
    }
    }''' % comp_id
    query = re.sub(r'\s+', ' ', query)
    try:
        res = requests.get('https://data.rcsb.org/graphql?query=' + query)
        smi = res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['SMILES_stereo']
        if smi is None:
            smi = res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['SMILES']
        if smi is None:
            m = Chem.MolFromInchi(res.json()['data']['chem_comp']['rcsb_chem_comp_descriptor']['InChI'])
            smi = Chem.MolToSmiles(m)
        assert smi is not None, "No reference smiles"
        return smi
    except:
        return ""


def regularize_binding_data(typ, sign, number, unit):
    
    # handle number that have uncertainty
    if '+-' in number:
        number = number.split('+-')[0]
    number = float(number)
    # handle sign
    sign = sign[1] + sign[0] if sign in ['=>', '=<'] else sign
    # convert Ka/Kb to Kd
    typ = typ.lower()
    if typ == 'ka' or typ == 'kb':
        typ = 'kd'
        assert unit.endswith('^-1'), f'Incorrect unit for Ka/Kb: {unit}'
        unit = unit.rstrip('^-1')
        number = 1 / number
    
    if unit == 'M' or unit == "":
        lognum = math.log10(number)
    elif unit == 'mM':
        lognum = math.log10(number) - 3
    elif unit == 'uM':
        lognum = math.log10(number) - 6
    elif unit == 'nM':
        lognum = math.log10(number) - 9
    elif unit == 'pM':
        lognum = math.log10(number) - 12
    elif unit == 'fM':
        lognum = math.log10(number) - 15
    else:
        lognum = None

    return {
        "measurement": typ,
        "sign": sign,
        "value": number,
        "unit": unit,
        "logvalue": lognum
    }

### Download BioLiP2 Dataset

In [ ]:
%%bash
wget https://zhanggroup.org/BioLiP/download/BioLiP.txt.gz
gunzip BioLiP.txt.gz

In [ ]:
%%bash
wget https://raw.githubusercontent.com/Mugsie1969/BindingMOAD/refs/heads/main/BindingMOAD_data_for_PDB_1_31_2022.csv

In [ ]:
columns = [
    'PDBID',
    'Receptor chain',
    'Resolution',
    'Binding site',
    'Ligand CCD',
    'Ligand chain',
    'Ligand serial number',
    'Binding site residues',
    'Binding site residues renumbered',
    'Catalytic site residues',
    'Catalytic site residues renumbered',
    'EC number',
    'GO terms',
    'Binding affinity (manual)',
    'Binding affinity (Binding MOAD)',
    'Binding affinity (PDBbind-CN)',
    'Binding affinity (Binding DB)',
    'UniProt ID',
    'PubMed ID',
    'Ligand residue sequence number',
    'Receptor sequence'
]
binding_cols = [
    'Binding affinity (manual)',
    'Binding affinity (Binding MOAD)',
    'Binding affinity (PDBbind-CN)',
    'Binding affinity (Binding DB)',
]
ions = [
    'MN', 'MG', 'ZN', 'NA', 'CO', 'CA', 'CU', 'NI', 'FE', 
    'HG', 'CE', 'AG', 'CD', 'CL', 'BR', 'F', 'XE', 'KR', 'AR',
    'K', 'LA', 'BA', 'SB', 'TL', 'CS', 'SR', 'AU', 'YB', 'GA', 'CR',
    'PD', 'MO', 'SE', 'LU', 'SM', 'PB', 'EU', 'PT', 'TB', 'RH', 'LI',
    'RB', 'RU', 'DY', 'RE', 'PR', 'OS', 'V', 'IR', 'ND', 'AL',
    'O', 'OH', 'SM'
]
biolip = pd.read_csv('BioLiP.txt', sep='\t', names=columns, low_memory=False, keep_default_na=False, na_values=[None, ""])
biolip

### Extract Binding data from BioLiP

In [ ]:
def process_biolip_binding_data(row):
    cols = [
        'Binding affinity (Binding MOAD)',
        'Binding affinity (manual)',
        'Binding affinity (PDBbind-CN)',
        'Binding affinity (Binding DB)',
    ]
    sources = ['MOAD', 'BioLiP', "PDBBind", "BindingDB"]
    type_rank = {
        "ec50": -1,
        "ic50": 0,
        "ki": 1,
        "kd": 2,
    }
    patt = re.compile(r"([a-zA-Z50]+)([~<>=]+)([\d.eE+-]+)([^\s,]*)")
    binding_data = None
    for col, src in zip(cols, sources):
        string = str(row[col])
        if string == 'nan':
            continue
        for match in re.findall(patt, string):
            tmp = regularize_binding_data(*match)
            if tmp['measurement'] not in ['kd', 'ki', 'ic50', 'ec50']:
                continue
            if binding_data is None:
                binding_data = tmp
            elif (type_rank[tmp['measurement']] > type_rank[binding_data['measurement']]):
                binding_data = tmp
        if binding_data:
            binding_data['source'] = src
            binding_data['origin'] = string
            break 
    if binding_data is not None and binding_data['logvalue'] > 3:
        print(row['PDBID'], 'has invalid biniding affinity:', binding_data['origin'], 'from', binding_data['source'])
        binding_data = None
    return binding_data


biolip_binding = biolip.dropna(
    subset=[
        'Binding affinity (manual)', 'Binding affinity (Binding MOAD)', 'Binding affinity (PDBbind-CN)', 'Binding affinity (Binding DB)'
    ],
    how='all'
)

biolip_binding_data = {}
for pdbid, subdf in biolip_binding.groupby('PDBID'):
    for _, row in subdf.iterrows():
        binding_data = process_biolip_binding_data(row)
        if binding_data:
            break
    if binding_data:
        biolip_binding_data[pdbid] = binding_data

biolip_binding_pdbids = list(biolip_binding_data.keys())
len(biolip_binding_pdbids)
# select_df = select_df.query('PDBID in @pdbid_with_binding')

In [ ]:
# some entries in MOAD is not correct
correction = {
    "2ok0": 'DT DC',
    "4zhh": "SM 4OL",
    "1jif": "CU BLM",
    "1npl": "MAN MAN"
}

moad = pd.read_csv('BindingMOAD_data_for_PDB_1_31_2022.csv')
moad['PDBID'] = moad['pdbFile'].str.lower()

biolip_binding_df = biolip.query('PDBID in @biolip_binding_pdbids')

hiq_sm, hiq_poly = [], []
for _, row in tqdm(moad.iterrows(), total=moad.shape[0]):
    pdbid = row['PDBID']
    binding = regularize_binding_data(
        row['measurement'],
        row['symbol'],
        str(row['magnitude']),
        row['units']
    )
    binding['source'] = 'MOAD'
    binding['origin'] = f"{row['measurement']}{row['symbol']}{row['magnitude']}{row['units']}"

    ccd = correction.get(pdbid, row['name'])
    
    if ' ' not in ccd:
        kind = 'sm'
    else:
        ccd = ccd.split()
        ccd_no_ions = [x for x in ccd if x not in ions]
        if len(ccd) - len(ccd_no_ions) > 0:
            print(pdbid, 'ligand contains ions:', ccd)
        if len(ccd_no_ions) == 1:
            kind = 'sm'
            ccd = ccd_no_ions[0]
        else:
            kind = 'poly'

    if kind == 'sm':
        # This is a small molecule
        if len(ccd) > 3:
            print(f'WARNING: {pdbid} ligand name ({ccd}) maybe incorrect')
        biolip_info = biolip_binding_df.query(f"PDBID == '{pdbid}' & `Ligand CCD` == '{ccd}'")
        if biolip_info.shape[0] > 0: # Entry in BioLiP
            for _, info in biolip_info.iterrows():
                data = {
                    'PDBID': pdbid, 'Ligand chain': info['Ligand chain'],
                    'Ligand CCD': ccd, 'Ligand residue sequence number': str(info['Ligand residue sequence number']).strip()
                }
                data.update(binding)
                hiq_sm.append(data)
        else: # Entry not in BioLiP
            data = {"PDBID": pdbid, 'Ligand chain': '', 'Ligand CCD': ccd, "Ligand residue sequence number": ''}
            data.update(binding)
            hiq_sm.append(data)
    else:
        # This is a polymer
        data = {
            "PDBID": pdbid, 'Ligand CCD': f'{ccd[0]}-{ccd[-1]}',
            'Ligand chain': '', 'Ligand residue sequence number': ''
        }
        data.update(binding)
        hiq_poly.append(data)


In [ ]:
# BioLiP but not in MOAD
moad_ids = moad['PDBID'].unique().tolist()
biolip_only = biolip_binding.query("PDBID in @biolip_binding_pdbids & PDBID not in @moad_ids & `Ligand CCD` not in @ions")

for pdbid, subdf in biolip_only.groupby("PDBID"):
    ccd = subdf['Ligand CCD'].unique().tolist()
    if len(ccd) > 1:
        print(f'{pdbid} has more than one residue: {ccd}')
    
    for _, row in subdf.iterrows():
        data = {"PDBID": pdbid}
        ccd = row['Ligand CCD']
        binding = process_biolip_binding_data(row)
        if binding is None:
            continue
        data.update(binding)
        data.update({
            'Ligand CCD': row['Ligand CCD'],
            'Ligand chain': row['Ligand chain'],
            'Ligand residue sequence number': str(row['Ligand residue sequence number']).strip()
        })
        if ccd in ['peptide', 'dna', 'rna']:
            hiq_poly.append(data)
        else:
            hiq_sm.append(data)

In [10]:
hiq_sm_df = pd.DataFrame(hiq_sm)
hiq_poly_df = pd.DataFrame(hiq_poly)
hiq_sm_df.sort_values('PDBID', inplace=True)
hiq_poly_df.sort_values('PDBID', inplace=True)
hiq_sm_df.to_csv('hiq_sm.csv', index=None)
hiq_poly_df.to_csv('hiq_poly.csv', index=None)

In [1]:
import pandas as pd

hiq_sm_df = pd.read_csv('hiq_sm.csv')
hiq_poly_df = pd.read_csv('hiq_poly.csv')

print("Before processing:")
print("Number of unique PDBIDs in HiQBind-sm:", hiq_sm_df['PDBID'].unique().shape[0])
print("Number of unique PDBIDs in HiQBind-poly:", hiq_poly_df['PDBID'].unique().shape[0])
print("Number of unique PDBIDs in HiQBind:", pd.concat((hiq_sm_df, hiq_poly_df))['PDBID'].unique().shape[0])

Before processing:
Number of unique PDBIDs in HiQBind-sm: 19105
Number of unique PDBIDs in HiQBind-poly: 1250
Number of unique PDBIDs in HiQBind: 20349
